In [25]:
from __future__ import annotations

import datetime
import math
import time
import xml.etree.ElementTree as ET
from common import load_csv, Route, Checkpoint, solar_power_out

In [26]:
#constants
CELL_AREA = 0.0153 #m^2

In [27]:
#absolute cell tilts 
tilts = {
    "hood_front": {
        "left_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "left_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "right_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "right_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44]
    },
    "top_front": {
        "leftmost_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "leftmost_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "leftmost_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "leftcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "leftcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "leftcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "rightcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "rightcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "rightcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "rightmost_top_4x3": [28.52, 28.52, 28.52, 28.52, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "rightmost_bottom_4x3": [22.16, 22.16, 22.16, 22.16, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14]
    },
    "top_back": {
        "leftmost_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "leftcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "rightcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "rightmost_4x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64, -3.64]
    },
    "back": {
        "leftmost_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightmost_top_4x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftmost_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64], 
        "rightcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightmost_bottom-4x4": [-4, -4, -4, -4, -5, -5, -5, -5, -6, -6, -6, -6]
    }
}

In [28]:
from astral import LocationInfo
from astral.sun import azimuth,elevation
class CellSolarData:
    def __init__(self, coord: Checkpoint, time: datetime.datetime, tilt: float):
        l = LocationInfo()
        l.name = f'{coord.lat},{coord.lon}'
        l.region = 'United States'
        l.timezone = 'US/Central' #update to be dynamic
        l.latitude = coord.lat
        l.longitude = coord.lon
        self.heading_azimuth_angle = coord.azimuth
        self.heading_azimuth_angle = 180 + self.heading_azimuth_angle if tilt < 0 else self.heading_azimuth_angle
        self.heading_azimuth_angle %= 360
        self.tilt = tilt * -1 if tilt < 0 else tilt
        self.lat = coord.lat
        self.lon = coord.lon
        self.elevation = coord.elevation / 1000 #convert to km
        self.time = time
        self.sun_elevation_angle = elevation(l.observer, time)
        self.sun_azimuth_angle = azimuth(l.observer, time)
        self.air_mass = 1/(math.cos(math.radians(90-self.sun_elevation_angle)) + 0.50572*(96.07995-(90-self.sun_elevation_angle))**-1.6364)
        self.incident_diffuse = 1.1*1.353*((1-0.14*(self.elevation / 1000))*0.7**self.air_mass**0.678 + 0.14*(self.elevation / 1000)) 
        self.cell_diffuse = self.incident_diffuse*(math.cos(math.radians(self.sun_elevation_angle))*math.sin(math.radians(self.tilt))*math.cos(math.radians(self.heading_azimuth_angle - self.sun_azimuth_angle)) + math.sin(math.radians(self.sun_elevation_angle))*math.cos(math.radians(self.tilt)))
        self.cell_irradiance = self.cell_diffuse * 1000 #convert to watts/m^2
        self.cloud_cover = coord.cloud_cover / 100
        cell_power_out = solar_power_out(self.cell_irradiance, self.cloud_cover) * CELL_AREA #watts
        self.cell_power_out = cell_power_out if cell_power_out > 0 else 0
    

In [29]:
def section_solar_power_out(coord: Checkpoint, time: datetime.datetime, section_tilts: dict):
    section_irradiance_sum = 0
    tilt_irradiances: dict[float, CellSolarData] = {}
    for array in section_tilts.keys():
        array_sum = 0
        for cell_angle in section_tilts[array]:
            if(cell_angle in tilt_irradiances):
                array_sum += tilt_irradiances[cell_angle].cell_power_out
            else:
                cell = CellSolarData(coord, time, cell_angle)
                
                # if(cell_angle<0):
                #     print(cell_angle)
                #     print(coord.azimuth)
                #     print(cell.__dict__)
                tilt_irradiances[cell_angle] = cell
                array_sum += cell.cell_power_out
        section_irradiance_sum += array_sum
    #watts
    return section_irradiance_sum

def total_solar_power_out(coord: Checkpoint, time: datetime.datetime, tilts: dict):
    # iterating over all of the sections in the car
    car_power_sum: float = 0
    for section in tilts.keys():
        car_power_sum += section_solar_power_out(coord, time, tilts[section])
    #watts
    return car_power_sum

    #sum of all the modules on the car
from pytz import timezone
est = timezone("US/Central")
utc = timezone("UTC")

def energy_captured_along_route_vconst(time_initial: datetime.datetime, velocity: float, route: Route):
    current_time = time_initial
    total_energy = 0
    velocity_ms = velocity / 3.6 #convert to m/2
    power_list = []
    for i in range(len(route.checkpoints)-1):
        segment_distance = route.checkpoints[i+1].distance if i==0 else route.checkpoints[i+1].distance - route.checkpoints[i].distance

        segment_time = segment_distance/velocity_ms

        total_power_in_current = total_solar_power_out(route.checkpoints[i], current_time, tilts=tilts)
        current_time += datetime.timedelta(seconds=segment_time)
        total_power_in_next = total_solar_power_out(route.checkpoints[i+1], current_time, tilts=tilts)
        total_power_in_avg = (total_power_in_current + total_power_in_next) / 2
        power_list.append(total_power_in_avg)
        total_energy += total_power_in_avg*segment_time #watt seconds
   
    print("times: ", time_initial.astimezone(est), current_time.astimezone(est), (current_time - time_initial).seconds / 60)
    print("avg power: ", total_energy / (current_time - time_initial).seconds)
    return (total_energy/3600, power_list) #in watt hours

def energy_captured_along_route(time_initial: datetime.datetime, velocities: list[float], route: Route):
    current_time = time_initial
    total_energy = 0
    
    power_list = []
    for i in range(len(route.checkpoints)-1):
        velocity_ms = velocities[i] / 3.6 #convert to m/2
        segment_distance = route.checkpoints[i+1].distance if i==0 else route.checkpoints[i+1].distance - route.checkpoints[i].distance

        segment_time = segment_distance/velocity_ms

        total_power_in_current = total_solar_power_out(route.checkpoints[i], current_time, tilts=tilts)
        current_time += datetime.timedelta(seconds=segment_time)
        total_power_in_next = total_solar_power_out(route.checkpoints[i+1], current_time, tilts=tilts)
        total_power_in_avg = (total_power_in_current + total_power_in_next) / 2
        power_list.append(total_power_in_avg)
        total_energy += total_power_in_avg*segment_time #watt seconds
   
    print("times: ", time_initial.astimezone(est), current_time.astimezone(est), (current_time - time_initial).seconds / 60)
    print("avg power: ", total_energy / (current_time - time_initial).seconds)
    return (total_energy/3600, power_list) #in watt hours


In [30]:
route = load_csv("A. Independence to Topeka")

In [31]:
#for independence to topeka, 2022
speed_limits = { #distance and speed in mph
    0: 20, #distance: speed limit
    1: 30,
    1.6: 35,
    7.1: 30,
    7.6: 25,
    7.9: 30,
    8.7: 40,
    16.7: 35,
    18.5: 45,
    20.1: 40,
    24.2: 35,
    24.5: 45,
    34.5: 50,
    38.5: 30,
    38.7: 50,
    43.3: 40,
    44.3: 30,
    44.6: 35,
    45.6: 40,
    46: 45,
    55.7: 55,
    60.4: 45,
    61.6: 55,
    69.6: 45,
    70.2: 65,
    76.6: 55,
    86.5: 45,
    87: 55,
    93.9: 45,
    94.9: 40,
    96: 30,
    97: 40,
    98: 30,
    98.25: 25
}

In [32]:
def average_velocity(route: Route, velocities: list[float]):
    times = []
    total_time = 0
    for (i,c) in enumerate(route.checkpoints):
        if i> 0:
            time = (c.distance - route.checkpoints[i-1].distance) / velocities[i-1]
            times.append(time)
            total_time += time
    v_avg = 0
    for (i,c) in enumerate(route.checkpoints):
        if i > 0:
            v_avg += velocities[i-1] * times[i-1] / total_time
    return v_avg
def assign_velocities_from_speed_limits(route: Route, speed_limits: dict):
    #assert average_velocity > 35 mph
    velocities = []
    j = 0
    speed_limit_keys = list(speed_limits.keys())
    for i in range(len(route.checkpoints) - 1):
        if (j < len(speed_limits.keys())):
            speed_limit_ends_at = speed_limit_keys[j+1] * 1.60934
            if(route.checkpoints[i].distance / 1000 >= speed_limit_ends_at):
                j+=1
        
        current_speed_limit = speed_limits[speed_limit_keys[j]] * 1.60934 #convert to km
        velocities.append(current_speed_limit)
    return velocities

velocities = assign_velocities_from_speed_limits(route, speed_limits)
print(velocities)
avg_v = average_velocity(route, velocities)
print(avg_v / 1.60934)

[32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 32.1868, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 48.2802, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 56.3269, 

In [33]:
loc_dt = datetime.datetime(2023, 6, 1, 8, 0, 0, tzinfo=est)
# print(loc_dt)
utc_dt = loc_dt.astimezone(utc)
tot_energy,powers = energy_captured_along_route_vconst(utc_dt, avg_v, route)
print(tot_energy)

times:  2023-06-01 08:51:00-05:00 2023-06-01 11:01:21.792726-05:00 130.35
avg power:  429.75714958884146
933.647407481758


In [168]:
import folium
import branca.colormap
import pandas as pd
import numpy as np
from itertools import zip_longest
def route_to_list(route: Route):
    coords: list[tuple[float, float]] = [] 
    for checkpoint in route.checkpoints:
        coords.append((checkpoint.lat, checkpoint.lon))
    return coords

def create_map(route: Route, power_list: list):
    coords = route_to_list(route)
    m = folium.Map()

    print(str(route.checkpoints[1]))
    colormap = branca.colormap.linear.YlOrRd_09.scale(min(power_list), max(power_list))
    tooltip = [f"{str(v)} | Power: {round(power_list[i-1])}" for (i,v) in enumerate(route.checkpoints)]
    for i in range(len(coords)-1):
        rgba = colormap.rgba_floats_tuple(power_list[i])
        rgba_scaled = (round(i*255) for i in rgba[0:3])
        # print('#{:02x}{:02x}{:02x}'.format(*rgba_scaled))
        folium.PolyLine(locations=[coords[i], coords[i+1]], tooltip=tooltip[i], weight=10, color='#{:02x}{:02x}{:02x}'.format(*rgba_scaled)).add_to(m)
    # folium.ColorLine(positions=coords, colormap=colormap, weight=5, colors=power_list).add_to(m)
    # for i,p in zip_longest(power_list, coords, fillvalue=np.mean(power_list)):
    #     folium.Marker(p, tooltip = i).add_to(m)
    df = pd.DataFrame(coords).rename(columns={0: 'lat', 1: 'lon'})
    print(df)
    sw = df[['lat', 'lon']].min().values.tolist()
    ne = df[['lat', 'lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    m.add_child(colormap)
    return m
m = create_map(route, powers)
m

Lat: 39.092184 | Lon: -94.417187 | Distance: 0.0 
| Azimuth: 271.5 | Elevation: 98.5 | Cloud Cover: 20 
| Wind Dir: 116.0 | Wind Speed: 6.0
            lat        lon
0     39.092185 -94.417077
1     39.092184 -94.417187
2     39.092197 -94.417811
3     39.092212 -94.418348
4     39.092216 -94.418486
...         ...        ...
2569  39.038138 -95.675183
2570  39.038337 -95.675887
2571  39.038268 -95.675943
2572  39.037843 -95.676128
2573  39.037758 -95.676164

[2574 rows x 2 columns]


In [18]:
VOLTAGE = 96 #V
def estimate_soc(q_initial: float, time_initial: datetime.time, velocity: float, route: Route):
    q_final = q_initial
    return q_final